In [1]:
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from datetime import datetime

# ---- ファイル名（例：請求書_20230520（YYYYMMDD）.xlsx）----
today = datetime.now()
filename = f"請求書_20230520（{today:%Y%m%d}）.xlsx"

# ---- ブック & シート ----
wb = Workbook()
ws = wb.active
ws.title = "請求書"

# ---- 列幅など ----
for col, w in {"A":3, "B":22, "C":12, "D":10, "E":12, "F":8, "G":16}.items():
    ws.column_dimensions[col].width = w
ws.row_dimensions[2].height = 28

# ---- スタイル ----
title_font = Font(name="Meiryo", size=18, bold=True)
hdr_font   = Font(name="Meiryo", size=12, bold=True)
bold_font  = Font(name="Meiryo", size=11, bold=True)
normal     = Font(name="Meiryo", size=11)
center = Alignment(horizontal="center", vertical="center")
left   = Alignment(horizontal="left",   vertical="center")
right  = Alignment(horizontal="right",  vertical="center")
thin   = Side(style="thin")
box    = Border(left=thin, right=thin, top=thin, bottom=thin)
th_fill= PatternFill("solid", fgColor="DDDDDD")

# ---- 見出し ----
ws.merge_cells("B2:D2")
ws["B2"] = "請求書"
ws["B2"].font, ws["B2"].alignment = title_font, left

# 会社情報
ws["B4"] = "株式会社ABC"
ws["B5"] = "〒101-0022 東京都千代田区神田練堀町300"
ws["B6"] = "TEL:03-1234-5678  FAX:03-1234-5678"
ws["B7"] = "担当者名:鈴木一郎 様"
ws["B4"].font = bold_font
for r in range(4,8): ws[f"B{r}"].alignment = left

# No. / 日付
ws["F4"], ws["G4"] = "No.", "0001"
ws["F5"], ws["G5"] = "日付", today.strftime("%Y/%m/%d")
ws["F4"].font = ws["F5"].font = bold_font
ws["F4"].alignment = ws["F5"].alignment = right

# ---- 明細ヘッダー ----
ws["B10"], ws["D10"], ws["E10"], ws["G10"] = "商品名", "数量", "単価", "金額"
for c in ["B10","D10","E10","G10"]:
    ws[c].font, ws[c].alignment, ws[c].fill, ws[c].border = hdr_font, center, th_fill, box

# ---- 明細行 ----
items = [("商品A",2,10000), ("商品B",1,15000)]
row0 = 11
for i,(name,qty,unit) in enumerate(items):
    r = row0 + i
    ws[f"B{r}"], ws[f"D{r}"], ws[f"E{r}"] = name, qty, unit
    ws[f"G{r}"] = f"=D{r}*E{r}"
    for col in ["B","D","E","G"]:
        ws[f"{col}{r}"].border = box
        ws[f"{col}{r}"].alignment = center if col in ["D","E","G"] else left

# 小計行のための合計（行13）
r = row0 + len(items)
ws[f"G{r}"] = f"=SUM(G{row0}:G{r-1})"
for col in ["B","D","E","G"]:
    ws[f"{col}{r}"].border = box
    ws[f"{col}{r}"].alignment = center

# ---- 小計・税・合計 ----
ws["B15"], ws["B16"], ws["B17"] = "小計", "消費税", "合計"
for c in ["B15","B16","B17"]:
    ws[c].font, ws[c].alignment = bold_font, left

ws["G15"] = f"=SUM(G{row0}:G{r})"  # 小計
ws["G16"] = "=ROUND(G15*0.10,0)"   # 税(10%)
ws["G17"] = "=G15+G16"            # 合計

for rr in range(15,18):
    for cc in ["B","C","D","E","F","G"]:
        ws[f"{cc}{rr}"].border = box
        ws[f"{cc}{rr}"].alignment = right if cc=="G" else left

# ---- 保存 ----
wb.save(filename)
print(f"Saved: {filename}")


Saved: 請求書_20230520（20250829）.xlsx
